In [1]:
import json
import requests
from pprint import pprint

In [5]:
# Define the names for the data source, skillset, index and indexer

datasource_name = "margies-custom-data"
skillset_name = "margies-custom-skillset-upd"
index_name = "margies-custom-index-upd"
indexer_name = "margies-custom-indexer-upd"

In [3]:
# Setup the endpoint

endpoint = 'https://ai102srch231123.search.windows.net'
headers = {'Content-Type': 'application/json',
           'api-key': 'NNqlehFz7Z3E63WnQROrOHvGQYPdUL5QsoSj8bRPkXAzSeClCxAd'}
params = {
    'api-version': '2020-06-30'
}

In [4]:
# Create a data source
# This data source points to your Azure Storage account.
# You should already have a blob container that contains the sample data
# The request should return a status code of 201 confirming success (204 - it already exists under the same name).

datasourceConnectionString = "DefaultEndpointsProtocol=https;AccountName=storacc231124;AccountKey=Fp7rRcxmXoVicTzXrcisAXubqP+22d96Sohk+CLx/9IKlWFGIMU4iLYxLfY9/HvV1kANz8roIJyV+AStHEPbpw==;EndpointSuffix=core.windows.net"

datasource_payload = {   
    "name" : datasource_name,  
    "description" : "Documents for search.",  
    "type" : "azureblob",
    "credentials" :
    { "connectionString" :  datasourceConnectionString
    },  
    "container" : { "name" : "margies" }} 

r = requests.put(endpoint + "/datasources/" + datasource_name,
                 data=json.dumps(datasource_payload), headers=headers, params=params)
print(r.status_code)

204


In [8]:
# Create a skillset
# The request should return a status code of 201 confirming success.

skillset_payload ={
    "name": skillset_name,
    "description": "Skillset for document extraction",
    "cognitiveServices": {
        "@odata.type": "#Microsoft.Azure.Search.CognitiveServicesByKey",
        "description": "Azure AI services",
        "key": "984f4248afcf469a9146c56607ba839e"
      },
    "skills": [{
        "name": "get-top-words",
        "@odata.type": "#Microsoft.Skills.Custom.WebApiSkill",
        "description": "custom skill to get top 10 most frequent words",
        "uri": "https://funcapp231124.azurewebsites.net/api/wordcount?code=dkrvFMEtlDN7d4ew_zNKaWZtEFOYorgwSkQfwq_ZSrXGAzFuPi3EUw==",
        "batchSize":1,
        "context": "/document",
        "inputs": [
          {
            "name": "text",
            "source": "/document/merged_content"
          },
          {
            "name":"language",
            "source": "/document/language"
          }  
        ],
        "outputs": [
            {
            "name": "text",
            "targetName": "topWords"
            }
        ]
      },{
        "@odata.type": "#Microsoft.Skills.Text.V3.SentimentSkill",
        "name": "get-sentiment",
        "description": "Evaluate sentiment",
        "context": "/document",
        "defaultLanguageCode": "en",
        "inputs": [
          {
            "name": "text",
            "source": "/document/merged_content"
          },
          {
            "name": "languageCode",
            "source": "/document/language"
          }
        ],
        "outputs": [
          {
            "name": "sentiment",
            "targetName": "sentimentLabel"
          }
        ]
      },
      {
        "@odata.type": "#Microsoft.Skills.Text.EntityRecognitionSkill",
        "name": "#1",
        "description": None,
        "context": "/document/merged_content",
        "categories": [
          "Person",
          "Quantity",
          "Organization",
          "URL",
          "Email",
          "Location",
          "DateTime"
        ],
        "defaultLanguageCode": "en",
        "minimumPrecision": None,
        "includeTypelessEntities": None,
        "inputs": [
          {
            "name": "text",
            "source": "/document/merged_content"
          },
          {
            "name": "languageCode",
            "source": "/document/language"
          }
        ],
        "outputs": [
          {
            "name": "locations",
            "targetName": "locations"
          },
          {
            "name": "entities",
            "targetName": "entities"
          }
        ]
      },
      {
        "@odata.type": "#Microsoft.Skills.Text.KeyPhraseExtractionSkill",
        "name": "#2",
        "description": None,
        "context": "/document/merged_content",
        "defaultLanguageCode": "en",
        "maxKeyPhraseCount": None,
        "inputs": [
          {
            "name": "text",
            "source": "/document/merged_content"
          },
          {
            "name": "languageCode",
            "source": "/document/language"
          }
        ],
        "outputs": [
          {
            "name": "keyPhrases",
            "targetName": "keyphrases"
          }
        ]
      },
      {
        "@odata.type": "#Microsoft.Skills.Text.LanguageDetectionSkill",
        "name": "#3",
        "description": None,
        "context": "/document",
        "inputs": [
          {
            "name": "text",
            "source": "/document/merged_content"
          }
        ],
        "outputs": [
          {
            "name": "languageCode",
            "targetName": "language"
          }
        ]
      },
      {
        "@odata.type": "#Microsoft.Skills.Text.MergeSkill",
        "name": "#4",
        "description": None,
        "context": "/document",
        "insertPreTag": " ",
        "insertPostTag": " ",
        "inputs": [
          {
            "name": "text",
            "source": "/document/content"
          },
          {
            "name": "itemsToInsert",
            "source": "/document/normalized_images/*/text"
          },
          {
            "name": "offsets",
            "source": "/document/normalized_images/*/contentOffset"
          }
        ],
        "outputs": [
          {
            "name": "mergedText",
            "targetName": "merged_content"
          }
        ]
      },
      {
        "@odata.type": "#Microsoft.Skills.Vision.OcrSkill",
        "name": "#5",
        "description": None,
        "context": "/document/normalized_images/*",
        "textExtractionAlgorithm": None,
        "lineEnding": "Space",
        "defaultLanguageCode": "en",
        "detectOrientation": True,
        "inputs": [
          {
            "name": "image",
            "source": "/document/normalized_images/*"
          }
        ],
        "outputs": [
          {
            "name": "text",
            "targetName": "text"
          },
          {
            "name": "layoutText",
            "targetName": "layoutText"
          }
        ]
      },
      {
        "@odata.type": "#Microsoft.Skills.Vision.ImageAnalysisSkill",
        "name": "#6",
        "description": None,
        "context": "/document/normalized_images/*",
        "defaultLanguageCode": "en",
        "visualFeatures": [
          "tags",
          "description"
        ],
        "details": [],
        "inputs": [
          {
            "name": "image",
            "source": "/document/normalized_images/*"
          }
        ],
        "outputs": [
          {
            "name": "tags",
            "targetName": "imageTags"
          },
          {
            "name": "description",
            "targetName": "imageCaption"
          }
        ]
      }
    ],
    "knowledgeStore": None,
    "encryptionKey": None
  }


r = requests.put(endpoint + "/skillsets/" + skillset_name,
                 data=json.dumps(skillset_payload), headers=headers, params=params)
print(r.status_code)

201


In [11]:
# Create an index
# Queries operate over the searchable fields and filterable fields in the index
# The request should return a status code of 201 confirming success.

index_payload ={
    "name": index_name,
    "fields": [
      {
        "name": "content",
        "type": "Edm.String",
        "facetable": False,
        "filterable": False,
        "retrievable": True,
        "searchable": True,
        "sortable": False
      },
      {
        "name": "metadata_storage_content_type",
        "type": "Edm.String",
        "facetable": False,
        "filterable": False,
        "retrievable": False,
        "searchable": False,
        "sortable": False
      },
      {
        "name": "metadata_storage_size",
        "type": "Edm.Int64",
        "facetable": False,
        "filterable": True,
        "retrievable": True,
        "sortable": True
      },
      {
        "name": "metadata_storage_last_modified",
        "type": "Edm.DateTimeOffset",
        "facetable": False,
        "filterable": True,
        "retrievable": True,
        "sortable": True
      },
      {
        "name": "metadata_storage_content_md5",
        "type": "Edm.String",
        "facetable": False,
        "filterable": False,
        "retrievable": False,
        "searchable": False,
        "sortable": False
      },
      {
        "name": "metadata_storage_name",
        "type": "Edm.String",
        "facetable": False,
        "filterable": True,
        "retrievable": True,
        "searchable": True,
        "sortable": True
      },
      {
        "name": "metadata_storage_path",
        "type": "Edm.String",
        "facetable": False,
        "filterable": False,
        "key": True,
        "retrievable": True,
        "searchable": False,
        "sortable": False
      },
      {
        "name": "metadata_storage_file_extension",
        "type": "Edm.String",
        "facetable": False,
        "filterable": False,
        "retrievable": False,
        "searchable": False,
        "sortable": False
      },
      {
        "name": "metadata_content_type",
        "type": "Edm.String",
        "facetable": False,
        "filterable": False,
        "retrievable": False,
        "searchable": False,
        "sortable": False
      },
      {
        "name": "metadata_language",
        "type": "Edm.String",
        "facetable": False,
        "filterable": False,
        "retrievable": False,
        "searchable": False,
        "sortable": False
      },
      {
        "name": "metadata_author",
        "type": "Edm.String",
        "facetable": True,
        "filterable": True,
        "retrievable": True,
        "searchable": True,
        "sortable": True
      },
      {
        "name": "metadata_title",
        "type": "Edm.String",
        "facetable": False,
        "filterable": False,
        "retrievable": False,
        "searchable": False,
        "sortable": False
      },
      {
        "name": "metadata_creation_date",
        "type": "Edm.DateTimeOffset",
        "facetable": False,
        "filterable": False,
        "retrievable": False,
        "sortable": False
      },
      {
        "name": "locations",
        "type": "Collection(Edm.String)",
        "facetable": False,
        "filterable": True,
        "retrievable": True,
        "searchable": True
      },
      {
        "name": "keyphrases",
        "type": "Collection(Edm.String)",
        "facetable": False,
        "filterable": False,
        "retrievable": True,
        "searchable": True
      },
      {
        "name": "language",
        "type": "Edm.String",
        "facetable": False,
        "filterable": True,
        "retrievable": True,
        "searchable": True,
        "sortable": False,
        "analyzer": "standard.lucene",
        "indexAnalyzer": None,
        "searchAnalyzer": None,
        "synonymMaps": [],
        "fields": []
      },
      {
        "name": "merged_content",
        "type": "Edm.String",
        "facetable": False,
        "filterable": False,
        "retrievable": True,
        "searchable": True,
        "sortable": False
      },
      {
        "name": "text",
        "type": "Collection(Edm.String)",
        "facetable": False,
        "filterable": False,
        "retrievable": True,
        "searchable": True
      },
      {
        "name": "layoutText",
        "type": "Collection(Edm.String)",
        "facetable": False,
        "filterable": False,
        "retrievable": True,
        "searchable": True
      },
      {
        "name": "imageTags",
        "type": "Collection(Edm.String)",
        "facetable": False,
        "filterable": False,
        "retrievable": True,
        "searchable": True
      },
      {
        "name": "imageCaption",
        "type": "Collection(Edm.String)",
        "facetable": False,
        "filterable": False,
        "retrievable": True,
        "searchable": True
      },
      {
        "name": "sentiment",
        "type": "Edm.String",
        "facetable": False,
        "filterable": True,
        "retrievable": True,
        "sortable": True
      },
      {
        "name": "url",
        "type": "Edm.String",
        "facetable": False,
        "filterable": True,
        "retrievable": True,
        "searchable": False,
        "sortable": False
      },
        {
      "name": "top_words",
      "type": "Collection(Edm.String)",
      "searchable": True,
      "sortable": False,
      "filterable": True,
      "facetable": False
      }]
  }


r = requests.put(endpoint + "/indexes/" + index_name,
                 data=json.dumps(index_payload), headers=headers, params=params)
print(r.status_code)

201


In [20]:
# Create an indexer

indexer_payload ={
    "name": indexer_name,
    "description": "Indexer for Margies Travel",
    "dataSourceName": "margies-custom-data",
    "skillsetName": "margies-custom-skillset-upd",
    "targetIndexName": "margies-custom-index-upd",
    "parameters": {
      "maxFailedItems": -1,
      "maxFailedItemsPerBatch": -1,
      "configuration": {
        "dataToExtract": "contentAndMetadata",
        "parsingMode": "default",
        "imageAction": "generateNormalizedImages"
      }
    },
    "fieldMappings": [
      {
        "sourceFieldName": "metadata_storage_path",
        "targetFieldName": "metadata_storage_path",
        "mappingFunction": {
          "name": "base64Encode"
        }
      },
      {
        "sourceFieldName": "metadata_storage_path",
        "targetFieldName": "url"
      }
    ],
    "outputFieldMappings": [
      {
        "sourceFieldName": "/document/merged_content/locations",
        "targetFieldName": "locations"
      },
      {
        "sourceFieldName": "/document/merged_content/keyphrases",
        "targetFieldName": "keyphrases"
      },
      {
        "sourceFieldName": "/document/language",
        "targetFieldName": "language"
      },
      {
        "sourceFieldName": "/document/merged_content",
        "targetFieldName": "merged_content"
      },
      {
        "sourceFieldName": "/document/normalized_images/*/text",
        "targetFieldName": "text"
      },
      {
        "sourceFieldName": "/document/normalized_images/*/layoutText",
        "targetFieldName": "layoutText"
      },
      {
        "sourceFieldName": "/document/normalized_images/*/imageTags/*/name",
        "targetFieldName": "imageTags"
      },
      {
        "sourceFieldName": "/document/normalized_images/*/imageCaption",
        "targetFieldName": "imageCaption"
      },
      {
        "sourceFieldName": "/document/sentimentLabel",
        "targetFieldName": "sentiment"
      },
      {
        "sourceFieldName" : "/document/topWords", 
        "targetFieldName" : "top_words"
      }
    ]
  }

    
r = requests.put(endpoint + "/indexers/" + indexer_name,
                 data=json.dumps(indexer_payload), headers=headers, params=params)
print(r.status_code)

201


In [21]:
# Getting indexer status to confirm that it's running

r = requests.get(endpoint + "/indexers/" + indexer_name +
                 "/status", headers=headers, params=params)
pprint(json.dumps(r.json(), indent=1))

('{\n'
 ' "@odata.context": '
 '"https://ai102srch231123.search.windows.net/$metadata#Microsoft.Azure.Search.V2020_06_30.IndexerExecutionInfo",\n'
 ' "name": "margies-custom-indexer-upd",\n'
 ' "status": "running",\n'
 ' "lastResult": null,\n'
 ' "executionHistory": [],\n'
 ' "limits": {\n'
 '  "maxRunTime": "PT0S",\n'
 '  "maxDocumentExtractionSize": 0,\n'
 '  "maxDocumentContentCharactersToExtract": 0\n'
 ' }\n'
 '}')


In [22]:
# Query the service for the index definition
# Query responses can be verbose. 
# If you get "Output exceeds the size limit. Open the full output data in a text editor", open the output in an editor.

r = requests.get(endpoint + "/indexes/" + index_name,
                 headers=headers, params=params)
pprint(json.dumps(r.json(), indent=1))

('{\n'
 ' "@odata.context": '
 '"https://ai102srch231123.search.windows.net/$metadata#indexes/$entity",\n'
 ' "@odata.etag": "\\"0x8DBECEDD0CDDE4C\\"",\n'
 ' "name": "margies-custom-index-upd",\n'
 ' "defaultScoringProfile": null,\n'
 ' "fields": [\n'
 '  {\n'
 '   "name": "content",\n'
 '   "type": "Edm.String",\n'
 '   "searchable": true,\n'
 '   "filterable": false,\n'
 '   "retrievable": true,\n'
 '   "sortable": false,\n'
 '   "facetable": false,\n'
 '   "key": false,\n'
 '   "indexAnalyzer": null,\n'
 '   "searchAnalyzer": null,\n'
 '   "analyzer": null,\n'
 '   "synonymMaps": []\n'
 '  },\n'
 '  {\n'
 '   "name": "metadata_storage_content_type",\n'
 '   "type": "Edm.String",\n'
 '   "searchable": false,\n'
 '   "filterable": false,\n'
 '   "retrievable": false,\n'
 '   "sortable": false,\n'
 '   "facetable": false,\n'
 '   "key": false,\n'
 '   "indexAnalyzer": null,\n'
 '   "searchAnalyzer": null,\n'
 '   "analyzer": null,\n'
 '   "synonymMaps": []\n'
 '  },\n'
 '  {\n'
 '   "n

In [26]:
# This query retrieves the **url**, and **wordcount** for all documents that mention *London* authored by *Reviewer* 
# that have a positive **sentiment** label (in other words, positive reviews that mention London)
# For keyword search, replace the asterisk with comma-separated query terms: search=microsoft,azure

r = requests.get(endpoint + "/indexes/" + index_name +
                 "/docs?&search=search=London&$select=url,top_words&$filter=metadata_author eq 'Reviewer' and sentiment eq 'positive'", 
                 headers=headers, params=params)
pprint(json.dumps(r.json(), indent=1))

('{\n'
 ' "@odata.context": '
 '"https://ai102srch231123.search.windows.net/indexes(\'margies-custom-index-upd\')/$metadata#docs(*)",\n'
 ' "value": [\n'
 '  {\n'
 '   "@search.score": 10.360352,\n'
 '   "url": '
 '"https://storacc231124.blob.core.windows.net/margies/reviews/201821.pdf",\n'
 '   "top_words": [\n'
 '    "great",\n'
 '    "hotel",\n'
 '    "london",\n'
 '    "tower",\n'
 '    "city",\n'
 '    "uk",\n'
 '    "stayed",\n'
 '    "nights",\n'
 '    "august"\n'
 '   ]\n'
 '  },\n'
 '  {\n'
 '   "@search.score": 9.17299,\n'
 '   "url": '
 '"https://storacc231124.blob.core.windows.net/margies/reviews/201855.pdf",\n'
 '   "top_words": [\n'
 '    "place",\n'
 '    "london",\n'
 '    "stayed",\n'
 '    "nights",\n'
 '    "great",\n'
 '    "stay",\n'
 '    "superb",\n'
 '    "buckingham",\n'
 '    "hotel"\n'
 '   ]\n'
 '  },\n'
 '  {\n'
 '   "@search.score": 6.0979753,\n'
 '   "url": '
 '"https://storacc231124.blob.core.windows.net/margies/reviews/201823.pdf",\n'
 '   "top_words": 

In [ ]:
# Azure Function App - creating a new custom wordcount function
# Replace the default function code with the following code:

# module.exports = async function (context, req) {
#      context.log('JavaScript HTTP trigger function processed a request.');
    
#      if (req.body && req.body.values) {
    
#          vals = req.body.values;
    
#          // Array of stop words to be ignored
#          var stopwords = ['', 'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 
#          "youre", "youve", "youll", "youd", 'your', 'yours', 'yourself', 
#          'yourselves', 'he', 'him', 'his', 'himself', 'she', "shes", 'her', 
#          'hers', 'herself', 'it', "its", 'itself', 'they', 'them', 
#          'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 
#          'this', 'that', "thatll", 'these', 'those', 'am', 'is', 'are', 'was',
#          'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 
#          'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 
#          'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 
#          'about', 'against', 'between', 'into', 'through', 'during', 'before', 
#          'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 
#          'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 
#          'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 
#          'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 
#          'only', 'own', 'same', 'so', 'than', 'too', 'very', 'can', 'will',
#          'just', "dont", 'should', "shouldve", 'now', "arent", "couldnt", 
#          "didnt", "doesnt", "hadnt", "hasnt", "havent", "isnt", "mightnt", "mustnt",
#          "neednt", "shant", "shouldnt", "wasnt", "werent", "wont", "wouldnt"];
    
#          res = {"values":[]};
    
#          for (rec in vals)
#          {
#              // Get the record ID and text for this input
#              resVal = {recordId:vals[rec].recordId, data:{}};
#              txt = vals[rec].data.text;
    
#              // remove punctuation and numerals
#              txt = txt.replace(/[^ A-Za-z_]/g,"").toLowerCase();
    
#              // Get an array of words
#              words = txt.split(" ")
    
#              // count instances of non-stopwords
#              wordCounts = {}
#              for(var i = 0; i < words.length; ++i) {
#                  word = words[i];
#                  if (stopwords.includes(word) == false )
#                  {
#                      if (wordCounts[word])
#                      {
#                          wordCounts[word] ++;
#                      }
#                      else
#                      {
#                          wordCounts[word] = 1;
#                      }
#                  }
#              }
    
#              // Convert wordcounts to an array
#              var topWords = [];
#              for (var word in wordCounts) {
#                  topWords.push([word, wordCounts[word]]);
#              }
    
#              // Sort in descending order of count
#              topWords.sort(function(a, b) {
#                  return b[1] - a[1];
#              });
    
#              // Get the first ten words from the first array dimension
#              resVal.data.text = topWords.slice(0,9)
#                .map(function(value,index) { return value[0]; });
    
#              res.values[rec] = resVal;
#          };
    
#          context.res = {
#              body: JSON.stringify(res),
#              headers: {
#              'Content-Type': 'application/json'
#          }
    
#          };
#      }
#      else {
#          context.res = {
#              status: 400,
#              body: {"errors":[{"message": "Invalid input"}]},
#              headers: {
#              'Content-Type': 'application/json'
#          }
    
#          };
#      }
#  };

In [ ]:
# In the Test/Run pane, replace the existing Body with the following JSON, 
# which reflects the schema expected by an Azure Cognitive Search skill

#  {
#      "values": [
#          {
#              "recordId": "a1",
#              "data":
#              {
#              "text":  "Tiger, tiger burning bright in the darkness of the night.",
#              "language": "en"
#              }
#          },
#          {
#              "recordId": "a2",
#              "data":
#              {
#              "text":  "The rain in spain stays mainly in the plains! That's where you'll find the rain!",
#              "language": "en"
#              }
#          }
#      ]
#  }